<a href="https://colab.research.google.com/github/LOANPIA/daTaSET/blob/main/Panda_ML_Tutorial_Hu%E1%BA%A5n_luy%E1%BB%87n_m%C3%B4_h%C3%ACnh_YOLOv5_v%E1%BB%9Bi_d%E1%BB%AF_li%E1%BB%87u_m%E1%BB%9Bi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Giới thiệu
Hiện nay, việc huấn luyện một mô hình phát hiện đối tượng (Object Detection) trên tập dữ liệu mới càng ngày càng trở nên dễ dàng. 

Hãy thử cùng tìm hiểu các bước huấn luyện một mô hình cho đối tượng mới dựa trên YOLOv5.
 


# Chuẩn bị dữ liệu
Đối tượng được dùng trong bài hướng dẫn này là gấu trúc (panda), nhưng các bước xây dựng mô hình có thể áp dụng với bất kỳ đối tượng nào khác. 

Để hiểu rõ hơn về bước này, các bạn có thể tham khảo bài chia sẻ về cách [Huấn luyện mô hình Object Detection với YOLOv5](https://hungk20.github.io/computer%20vision/practical/train-yolov5/).
## Tập hợp ảnh và gắn nhãn
Có vài cách để thu thập ảnh cho việc huấn luyện mô hình:
- Tìm trên [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html), đây là tập hợp hơn 9 triệu ảnh với 6000 nhóm khác nhau.
- Tìm với công cụ tìm kiếm (Google, Bing, …) với lưu ý cần kiểm tra kỹ về bản quyền sử dụng ảnh.
- Trích xuất các khung hình từ Videos
- Tự chụp ảnh
- ...

Sau đó, ảnh cần được gắn nhãn theo định dạng của YOLO. Mỗi ảnh sẽ có một nhãn (file txt) tương ứng để chứa thông tin về đường bao của đối tượng:
- Mỗi dòng chứa thông tin của một đối tượng
- Mỗi dòng có 5 giá trị chứa thông tin về đối tượng và toạ độ đường bao: đối tượng, toạ độ trung tâm x, toạ độ trung tâm y, chiều rộng, chiều dài.
- Lưu ý là toạ độ đường bao đã được chuẩn hoá về khoảng (0, 1) (đây là yêu cầu của mô hình YOLO)

## Sắp xếp thư mục ảnh và nhãn
Sau khi đã có ảnh và nhãn, tập dữ liệu có thể được sắp xếp như sau:
```
custom_dataset
├── custom_dataset.yaml
├── custom_model.yaml
└── images_and_labels
```
- File cấu hình của tập dữ liệu `custom_dataset.yaml` chứa đường dẫn đến thư mục ảnh và thông tin đối tượng.
```python
# relative paths from folder yolov5
#  .
#  ├── custom_dataset
#  └── yolov5
train: ../custom_dataset/images_and_labels/images/train/
val: ../custom_dataset/images_and_labels/images/valid/
# number of classes
nc: 1
# class names
names: ['panda']
```
- File cấu hình của mô hình `custom_model.yaml` chứa thông tin cấu trúc mạng, ở đây chúng ta dùng cấu trúc mô hình YOLOv5s (cỡ nhỏ) với mục tiêu rút ngắn thời gian huấn luyện và kích thước mô hình. Lưu ý là chúng ta hoàn toàn có thể dùng cấu trúc khác, ví dụ như YOLOv5m, YOLOv5l, YOLOv5x từ YOLOv5 hoặc một cấu trúc mới do chúng ta tự thiết kế.
```python
# custom config based on yolov5s, just change number of class
nc: 1  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
...
```
- Thư mục `image_and_labels` chứa ảnh và nhãn dùng cho việc huấn luyện mô hình.
```
images_and_labels
├── images
│   ├── train
│   │   ├── train_001.jpg
│   │   ├── train_002.jpg
│   │   └── ...
│   ├── valid
│   │   ├── valid_001.jpg
│   │   ├── valid_002.jpg
│   │   └── ... 
│   └── test
│       ├── test_001.jpg
│       ├── test_002.jpg
│       └── ...
└── labels
  ├── train
  │   ├── train_001.txt
  │   ├── train_002.txt
  │   └── ...
  └── valid
      ├── valid_001.txt
      ├── valid_002.txt
      └── ...
```

Với tập dữ liệu về gấu trúc, các bạn có thể tải về từ [Panda ML Blog Github](https://github.com/hungk20/PandaML/blob/master/object_detection/train_yolo/custom_dataset.zip) về tải lên Colab. Các bạn cũng hoàn toàn có thể dùng dữ liệu của mình để dùng với notebook này (chỉ cần chuẩn bị đúng các file cấu hình là ổn).

In [ ]:
# unzip thư mục ảnh và nhãn cùng với các files cấu hình
!unzip -q custom_dataset.zip

In [ ]:
# kiểm tra file cấu hình của tập dữ liệu
!cat custom_dataset/custom_dataset.yaml

In [ ]:
# kiểm tra file cấu hình của mô hình
!head custom_dataset/custom_model.yaml

# Huấn luyện mô hình
## Cài đặt thư viện liên quan
Tải YOLOv5 từ GitHub và các cài đặt các thư viện liên quan.


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
# vào thư mục yolov5 mới được cloned về
%cd yolov5
# (không bắt buộc) dùng 1 commit nhất định
# !git reset --hard 886f1c03d839575afecb059accf74296fad395b6
# cài đặt các thư viện cần thiết
!pip install -qr requirements.txt  # install dependencies

In [ ]:
import torch

# kiểm tra notebook đã dùng GPU (thời gian huấn luyện nhanh hơn)
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Lưu ý là đến bước này thì chúng ta đang ở trong thư mục `yolov5`, thư mục này ở cạnh thư mục `custom_dataset` như sau:
```
.
├── custom_dataset
└── yolov5
```

## Huấn luyện
Các bước chuẩn bị đã xong, giờ việc huấn luyện trở nên rất đơn giản với `train.py` trong thư mục `yolov5`.

Một số tham số được dùng:
- `img`: kích thước ảnh (độ phân giải)
- `batch`: số ảnh dùng để huấn luyện trong mỗi lượt
- `epochs`: số lượt huấn luyện cho tất cả các ảnh trong tập dữ liệu train
- `data`: đường dẫn đến file cấu hình của tập dữ liệu
- `cfg`: đường dẫn đến file cấu hình của mô hình
- `weights`: đường dẫn đến file weight chứa độ liên kết giữa các neuron (để '' là để huấn luyện từ đầu)
- `name`: tên thư mục để lưu mô hình
- `cache`: dùng bộ nhớ đệm để huấn luyện nhanh hơn

In [ ]:
%%time
!python train.py --img 416 --batch 16 --epochs 500 \
  --data ../custom_dataset/custom_dataset.yaml \
  --cfg ../custom_dataset/custom_model.yaml \
  --weights '' --name custom_model  --cache

## Kết quả


Ảnh đầu gốc vào và ảnh tăng cường (data augmentaion) khi huấn luyện mô hình được lưu ở `yolov5/runs/train/custom_model/`.

In [ ]:
from IPython.display import Image, display

print("Ảnh gốc và nhãn tương ứng:")
Image(filename='runs/train/custom_model/val_batch0_labels.jpg', width=900)

In [ ]:
print("Ảnh tăng cường (augmented)")
Image(filename='runs/train/custom_model/train_batch0.jpg', width=900)

Dùng `tensorboard` để kiểm tra kết quả của việc huấn luyện (các chỉ số thay đổi như thế nào sau mỗi epoch).

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# Dự đoán với mô hình được huấn luyện

Khi việc huấn luyện kết thúc, có 2 file weights mô hình được lưu lại:

- Mô hình tốt nhất: `best.pt` (tốt nhất dựa trên chỉ số Average Precision trên tập kiểm định)
- Mô hình cuối cùng: `last.pt`

In [ ]:
!ls runs/train/custom_model/weights

Chúng ta dùng `detect.py` để thực hiện dự đoán với mô hình tốt nhất có được từ quá trình huấn luyện.
- `weights`: file weight dùng để dự đoán
- `img`: kích thước ảnh (độ phân giải)
- `conf`: độ tin cậy
- `source`: đường dẫn đến thư mục ảnh

In [ ]:
!python detect.py --weights runs/train/custom_model/weights/best.pt \
    --img 416 --conf 0.4 \
    --source ../custom_dataset/images_and_labels/images/test

Kết quả dự đoán được lưu tại thư mục `/yolov5/runs/detect/exp/` (hoặc exp1, 2, 3, … tuỳ vào số lần chúng ta chạy dự đoán với `detect.py`).

In [ ]:
import glob

for imageName in glob.glob('runs/detect/exp/*.jpg'):
    display(Image(filename=imageName))
    print("\n")

# Lưu mô hình cho những lần sử dụng sau
Các files trên Colab sẽ bị xoá sau một khoảng thời gian nhất định, chúng ta nên lưu lại files mô hình để sử dụng về sau. Các files có thể được tải trực tiếp về máy hoặc lưu lại trên Google Drive.
```
from google.colab import drive
drive.mount('/content/gdrive')
```

```
%cp /content/yolov5/runs/train/custom_model/weights/best.pt /content/gdrive/Your_Drive
```



# Tạm kết

Bài hôm nay đã giới thiệu đến các bạn các bước để huấn luyện một mô hình Object Detection dựa trên YOLOv5. Có lẽ bước tốn thời gian nhất là bước tìm ảnh phù hợp và gắn nhãn cho ảnh. Còn các bước huấn luyện và dự đoán thì khá dễ dàng. Trong thực tế cũng vậy, bước chuẩn bị dữ liệu (data preparation) thường cũng là bước tốn thời gian nhất.

Hy vọng là với bài hướng dẫn này, các bạn cũng sẽ huấn luyện được những mô hình thú vị cho riêng mình.

-- Panda ML Blog

# Tham khảo
- [YOLOv5 repository](https://github.com/ultralytics/yolov5)
- [Offical YOLOv5 Tutorial by Ultralytics](https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb)
- [Train YOLOv5 blog by RoboFlow](https://blog.roboflow.ai/how-to-train-yolov5-on-a-custom-dataset/)
- [Train YOLOv5 Tutorial by RoboFlow](https://colab.research.google.com/drive/1gDZ2xcTOgR39tGGs-EZ6i3RTs16wmzZQ)
- [How to Create an End to End Object Detector using Yolov5](https://towardsdatascience.com/how-to-create-an-end-to-end-object-detector-using-yolov5-35fbb1a02810)